<a href="https://colab.research.google.com/github/Emmahgithub/Administraci-n-De-Riesgos-Financieros/blob/main/Riesgo_Historico_ultimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OJO SE DEBEN CARGAR LAS RUTAS DE LOS ARCHIVOS .TEX DE LAS CURVAS

In [100]:
# ============================================================
# CUADRO 0: IMPORTS Y PARÁMETROS GLOBALES
# ============================================================

!git clone https://github.com/Emmahgithub/Administraci-n-De-Riesgos-Financieros.git


import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from math import log, sqrt, exp, isfinite
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# FECHAS Y PARÁMETROS GENERALES
# -----------------------------

VAL_DATE_STR = "2025-09-08"         # Fecha de valoración (YYYY-MM-DD)
VAL_DATE = pd.Timestamp(VAL_DATE_STR)

ALPHA = 0.99                         # Nivel de confianza para VaR/CVaR
HORIZON_N_DAYS = 1                   # Horizonte de riesgo "n" días (default 1)
LOOKBACK_DAYS = 1000                 # Ventana histórica base (ajustable)
EWMA_LAMBDA = 0.94                   # Parámetro de alisado EWMA (diario)

# Base día: 360 para instrumentos MXN (convención simple de clase)
DAY_COUNT = 360.0

# -----------------------------
# TICKERS (yfinance)
# -----------------------------
# Acciones (BMV) - *exactamente como pediste*
SYMBOLS_EQ = ["AMXB.MX", "GCARSOA1.MX", "WALMEX.MX"]

# FX (Yahoo Finance)
TICK_USDMXN = "MXN=X"                # spot USD/MXN
TICK_EURMXN = "EURMXN=X"
TICK_BTCUSD = "BTC-USD"

# -----------------------------
# MULTIPLICADORES DE FUTUROS (ajusta si tu profesor usa otros)
# -----------------------------
USDMXN_FUT_MULT   = 10000.0   # típico MexDer (10,000 USD/contrato)
WALMEX_FUT_MULT   = 100.0     # típico 100 acciones/contrato

# -----------------------------
# SEED (por reproducibilidad cuando aplique)
# -----------------------------
np.random.seed(42)


fatal: destination path 'Administraci-n-De-Riesgos-Financieros' already exists and is not an empty directory.


In [101]:
# ============================================================
# CUADRO 1: CURVAS, INTERPOLACIÓN ALAMBRADA Y DESCUENTO  (FIX)
#  - Lectura robusta de tfondeo.txt: ya NO asume columna 'fecha'
#  - Detecta automáticamente columna de fecha y de tasa
# ============================================================



def _read_curve_txt(path: str) -> pd.DataFrame:
    """
    Lee archivos .txt de curvas con encabezado: DATE, 1, 7, 30, 90, ...
    Devuelve DataFrame indexado por fecha (pd.Timestamp) y columnas int (tenores en días).
    Asume que las tasas vienen en % anual.
    """
    # Acepta separadores comunes (espacios, tabs, coma, punto y coma, pipe)
    df = pd.read_csv(path, sep=r"[,\s;|]+", engine="python")
    # Normalizar nombre de la primera columna como DATE
    first_col = df.columns[0]
    df = df.rename(columns={first_col: "DATE"})
    # Parsear fecha
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=False)
    df = df.dropna(subset=["DATE"]).set_index("DATE").sort_index()

    # Convertir encabezados de tenores a int (si algún encabezado no es numérico, lo descarta)
    tenors = []
    vals = []
    for c in df.columns:
        try:
            tenors.append(int(str(c).strip()))
            vals.append(df[c].astype(float))
        except Exception:
            # si no se puede convertir el encabezado a entero, se ignora esa columna
            continue
    if not tenors:
        raise ValueError(f"No se encontraron columnas de tenor numéricas en {path}")
    out = pd.DataFrame({int(t): v for t, v in zip(tenors, vals)}, index=df.index)
    out = out.sort_index(axis=1)
    return out

def _read_tfondeo(path: str) -> pd.DataFrame:
    """
    Lee tfondeo (serie diaria) sin asumir nombres de columna.
    - Detecta automáticamente la columna de FECHA (la que mejor parsea a datetime).
    - Detecta automáticamente la columna de TASA (primera numérica con mayor cobertura).
    Devuelve DataFrame con índice de fecha y UNA columna con tenor 28 días (en % anual).
    """
    raw = pd.read_csv(path, sep=r"[,\s;|]+", engine="python")
    if raw.shape[1] < 2:
        # intento sin separador (CSV simple)
        raw = pd.read_csv(path)

    # 1) Encontrar columna de fecha
    best_col = None
    best_ok = -1
    for c in raw.columns:
        s = pd.to_datetime(raw[c], errors="coerce", dayfirst=False)
        ok = s.notna().sum()
        if ok > best_ok:
            best_ok = ok
            best_col = c
    if best_col is None or best_ok <= 0:
        raise ValueError("No se pudo identificar columna de fecha en tfondeo.")
    dates = pd.to_datetime(raw[best_col], errors="coerce", dayfirst=False)

    # 2) Encontrar columna de tasa (numérica con mayor cobertura)
    rate_col = None
    best_n = -1
    for c in raw.columns:
        if c == best_col:
            continue
        s = pd.to_numeric(raw[c], errors="coerce")
        n = s.notna().sum()
        if n > best_n:
            best_n = n
            rate_col = c
    if rate_col is None or best_n <= 0:
        raise ValueError("No se encontró columna de tasa en tfondeo.")

    df = pd.DataFrame({"DATE": dates, "RATE": pd.to_numeric(raw[rate_col], errors="coerce")})
    df = df.dropna(subset=["DATE", "RATE"]).set_index("DATE").sort_index()

    # Construimos una "curva" mínima con tenor 28 días (para cupones Bondes)
    out = pd.DataFrame(index=df.index)
    out[28] = df["RATE"].astype(float)  # en % anual (consistente con el resto de curvas)
    return out

# Leer curvas desde los .txt
PATH_GUBER   = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_guber.txt"
PATH_DIRS    = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_DIRS_SW_OP.txt"
PATH_LIBOR   = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_libor.txt"
PATH_FONDEO  = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tfondeo.txt"
PATH_TIEE  = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tvoltiie_opc.txt"
PATH_FORWARD  = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_fwd.txt"
PATH_T_SW = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_TIIE_SW_OP.txt"
PATH_DIRS_SW = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_DIRS_SW_OP.txt"
PATH_YIELD = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_yield.txt"
PATH_GUBER_ST  = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_guber_st.txt"
PATH_TVOLTIEE_OPC = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tvoltiie_opc.txt"




curve_guber  = _read_curve_txt(PATH_GUBER)    # CETES/Guber (MXN)
curve_dirs   = _read_curve_txt(PATH_DIRS)     # TIIE/DIRS (MXN)
curve_libor  = _read_curve_txt(PATH_LIBOR)    # USD Libor (proxy)
curve_fondeo = _read_tfondeo(PATH_FONDEO)     # Fondeo (MXN, tenor 28d)
curve_tiie = _read_tfondeo(PATH_TIEE)
curve_fwd = _read_curve_txt(PATH_FORWARD)
curve_tiie_sw = _read_curve_txt(PATH_T_SW)
curve_dirs_sw = _read_curve_txt(PATH_DIRS_SW)
curve_yield = _read_curve_txt(PATH_YIELD)
curve_guber_st = _read_curve_txt(PATH_GUBER_ST)
curve_tvoltiie_opc = _read_tfondeo(PATH_TVOLTIEE_OPC)
curve_ones = pd.DataFrame({'ones': [1.0] * len(curve_guber.index)}, index=curve_guber.index)


DAY_COUNT = 360.0  # definir aquí si este cuadro se ejecuta aislado

def alambrada_rate_at(df_curve_for_date: pd.Series, t_days: int) -> float:
    """
    Interpolación alambrada (lineal) sobre TENOR (días) de la TASA (% anual).
    - Extremos: extrapola linealmente usando el primer/par de puntos del extremo.
    """
    ten = np.array(df_curve_for_date.index, dtype=int)
    y   = np.array(df_curve_for_date.values, dtype=float)
    if len(ten) == 0:
        return 0.0

    if t_days <= ten[0]:
        if len(ten) == 1:
            return float(y[0])
        slope = (y[1] - y[0]) / (ten[1] - ten[0])
        return float(y[0] + slope * (t_days - ten[0]))

    if t_days >= ten[-1]:
        if len(ten) == 1:
            return float(y[-1])
        slope = (y[-1] - y[-2]) / (ten[-1] - ten[-2])
        return float(y[-1] + slope * (t_days - ten[-1]))

    i = np.searchsorted(ten, t_days)
    t0, t1 = ten[i-1], ten[i]
    y0, y1 = y[i-1], y[i]
    w = (t_days - t0) / (t1 - t0)
    return float(y0 * (1 - w) + y1 * w)

def alambrada_df_func(curve_panel: pd.DataFrame, when: pd.Timestamp):
    """
    Devuelve una función df(t_days) basada en la curva del 'when' (fecha exacta más cercana <= when).
    Convierte tasa % anual simple a DF con base 360: DF = 1 / (1 + r * t), con r en decimal y t en años 360.
    """
    # Selección de fila más reciente <= when
    if when not in curve_panel.index:
        ix = curve_panel.index.searchsorted(when)
        ix = max(0, ix - 1)
        row = curve_panel.iloc[ix]
    else:
        row = curve_panel.loc[when]
    row = row.dropna()

    def df(t_days: int) -> float:
        r_pct = alambrada_rate_at(row, max(int(t_days), 1))  # % anual
        r = r_pct / 100.0
        t = max(t_days, 0) / DAY_COUNT
        return 1.0 / (1.0 + r * t)
    return df

def forward_rate_from_curve(curve_panel: pd.DataFrame, when: pd.Timestamp, start_days: int, tau_days: int) -> float:
    """
    Tasa forward simple entre [start_days, start_days+tau_days], derivada de curva de descuento.
    """
    df = alambrada_df_func(curve_panel, when)
    t0 = max(start_days, 0)
    t1 = t0 + max(tau_days, 1)
    D0, D1 = df(t0), df(t1)
    tau = (t1 - t0) / DAY_COUNT
    fwd = (D0 / D1 - 1.0) / max(tau, 1e-9)
    return float(fwd)

In [102]:
# ============================================================
# CUADRO 2 (FIX): HISTÓRICOS DE PRECIOS (YF) Y CÁLCULO DE SHOCKS
#  - Soporta 'Close' (auto_adjust=True) o 'Adj Close' (auto_adjust=False)
#  - Soporta DataFrame con MultiIndex (OHLCV por campo) o columnas planas
#  - Construye BTCMXN = (BTC-USD) * (USDMXN)
# ============================================================

def _last_business_before(t: pd.Timestamp, series: pd.Series) -> pd.Timestamp:
    """ Devuelve la última fecha <= t presente en la serie. """
    idx = series.index
    if t in idx:
        return t
    pos = idx.searchsorted(t)
    pos = max(0, pos - 1)
    return idx[pos]

def _extract_close_frame(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extrae una matriz de precios de cierre por ticker desde el DataFrame crudo de yf.download.
    Prioriza 'Close'; si no existe, intenta 'Adj Close'; si tampoco, toma la primera capa disponible.
    Devuelve DataFrame (index fechas, columnas = tickers).
    """
    if isinstance(df.columns, pd.MultiIndex):
        # Primer nivel suelen ser campos: 'Open','High','Low','Close','Adj Close','Volume'
        lvl0 = df.columns.get_level_values(0)
        if "Close" in set(lvl0):
            close = df["Close"].copy()
        elif "Adj Close" in set(lvl0):
            close = df["Adj Close"].copy()
        else:
            # fallback: toma el primer campo disponible
            first_field = lvl0[0]
            close = df[first_field].copy()
        return close
    else:
        # Columnas planas (yfinance a veces devuelve directamente columnas por ticker)
        # Si detectamos columnas de OHLC, intentamos 'Close'; si no, asumimos que ya son tickers.
        possible_fields = {"Open","High","Low","Close","Adj Close","Volume"}
        has_fields = any((c in possible_fields) for c in map(str, df.columns))
        if has_fields:
            if "Close" in df.columns:
                return df[["Close"]].rename(columns={"Close": "CLOSE_SINGLE"})
            elif "Adj Close" in df.columns:
                return df[["Adj Close"]].rename(columns={"Adj Close": "CLOSE_SINGLE"})
            else:
                # fallback: usa todo lo que haya
                return df.copy()
        else:
            # Parece que las columnas ya son tickers => retornamos tal cual
            return df.copy()

def download_hist_prices(lookback_days=LOOKBACK_DAYS, end_date=VAL_DATE):
    """
    Descarga precios diarios para acciones y FX/BTC.
    Devuelve DataFrame con columnas = tickers y filas = fechas (cierre ajustado).
    """
    start = (end_date - pd.Timedelta(days=lookback_days*2)).strftime("%Y-%m-%d")
    end   = (end_date + pd.Timedelta(days=3)).strftime("%Y-%m-%d")

    tickers = SYMBOLS_EQ + [TICK_USDMXN, TICK_EURMXN, TICK_BTCUSD]
    raw = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)
    if raw is None or len(raw) == 0:
        raise RuntimeError("yfinance no devolvió datos. Revisa conexión/fechas/tickers.")

    close = _extract_close_frame(raw)

    # Si _extract_close_frame devolvió una sola columna 'CLOSE_SINGLE' (caso raro 1 ticker),
    # la renombramos con el ticker correspondiente:
    if close.shape[1] == 1 and close.columns[0] == "CLOSE_SINGLE":
        # Si pedimos varios tickers pero yf solo devolvió uno, mantenemos ese nombre
        # (no siempre es posible mapearlo sin la info de columnas originales).
        close = close.rename(columns={"CLOSE_SINGLE": tickers[0]})

    # Limpiar y forward-fill
    close = close.dropna(how="all").ffill()

    # Asegurar que todas las columnas que pedimos existan (si alguna falta, la intentamos crear vacía)
    for tk in tickers:
        if tk not in close.columns:
            close[tk] = np.nan
    close = close[tickers]

    # BTC en MXN = BTC-USD * USDMXN
    btc_mxn = close[TICK_BTCUSD] * close[TICK_USDMXN]
    close = close.assign(BTCMXN=btc_mxn)

    # Cortar ventana a LOOKBACK_DAYS exactos respecto a la última fecha disponible antes de VAL_DATE
    if close[TICK_USDMXN].dropna().empty:
        raise RuntimeError("No hay datos de MXN=X para alinear la ventana histórica.")
    last = _last_business_before(end_date, close[TICK_USDMXN].dropna())
    close = close.loc[:last].tail(lookback_days + 2).dropna(how="all")

    # En algunos casos, alguna serie puede iniciar después; hacemos ffill de nuevo y limpiamos filas vacías
    close = close.ffill().dropna()

    # Asegurar índice tipo fecha ordenado
    close = close.sort_index()
    return close

def build_spot_panel():
    """
    Construye panel de niveles 'spot' para:
      - Acciones BMV: AMXB.MX, GCARSOA1.MX, WALMEX.MX
      - FX: USDMXN (MXN por USD), EURMXN (MXN por EUR)
      - BTC en MXN: BTC-USD * USDMXN
    """
    px = download_hist_prices()
    return px

def make_returns_for_horizon(series: pd.Series, n_days: int):
    """
    Retornos multiplicativos para horizonte n días: R_t = S_t / S_{t-n} - 1, alineados al t (fin del periodo).
    """
    return series / series.shift(n_days) - 1.0

def apply_price_shock(S0: float, r: float) -> float:
    """ Aplica shock multiplicativo al nivel: S' = S0 * (1 + r) """
    return float(S0 * (1.0 + r))


In [103]:
# ============================================================
# CUADRO 3 (ACTUALIZADO): PORTAFOLIO EXACTO DEL ENUNCIADO
# ============================================================

# Si los multiplicadores YA están definidos en otro cuadro, se respetan.
try:
    USDMXN_FUT_MULT
except NameError:
    USDMXN_FUT_MULT = 1000.0   # default; ajusta si tu mercado usa otro

try:
    WALMEX_FUT_MULT
except NameError:
    WALMEX_FUT_MULT = 1000.0   # default; 1000 suele cuadrar con tus MTM

PORT = {
    # ------------------------- Acciones BMV -------------------------
    # (nota: qty < 0 => short)
    "eq_GCARSO": {"type": "equity", "ticker": "GCARSOA1.MX", "qty":  1000.0},
    "eq_AMXB"  : {"type": "equity", "ticker": "AMXB.MX",     "qty": -10000.0},
    "eq_WALMEX": {"type": "equity", "ticker": "WALMEX.MX",   "qty":  3000.0},

    # ------------------------- Divisas y cripto ---------------------
    # USD y EUR con posición real; GBP en 0.0 para que aparezca en la tabla.
    "fx_USD"   : {"type": "fx",     "ticker": "MXN=X",     "qty": 1500.0},    # USD/MXN
    "fx_EUR"   : {"type": "fx",     "ticker": "EURMXN=X",  "qty": 1200.0},    # EUR/MXN
    # BTC cotiza en USD; lo convertimos a MXN multiplicando por USD/MXN al día:
    "cr_BTC"   : {"type": "crypto", "ticker": "BTC-USD",   "qty": 0.025,
                  "convert_to_mxn": True, "fx_link": "MXN=X"},

    # ------------------------- Bonos -------------------------------
    "bono_CETES_ZC_180": {"type": "bond_zc",  "curve": "guber",
                          "notional": 15000.0, "ttm_days": 180},
    "bonoM_11p5_3600"  : {"type": "bond_fix", "curve": "guber",
                          "notional": 1300.0, "coupon": 0.115,
                          "ttm_days": 3600, "freq_days": 182},

    # BONDE (flotante 28d) corto 1200: cupón por fondeo y descuento por guber
    "bondes_1200_707d_g" : {"type": "bonde",
                          "notional": -1200.0, "ttm_days": 707, "coupon_freq_days": 28,
                          "curve_cpn": "fondeo", "curve_disc": "guber"},
    "bondes_1200_707d_f" : {"type": "bonde",
                          "notional": -1200.0, "ttm_days": 707, "coupon_freq_days": 28,
                          "curve_cpn": "fondeo", "curve_disc": "fondeo"},
    "bondes_1200_707d_s" : {"type": "bonde",
                          "notional": -1200.0, "ttm_days": 707, "coupon_freq_days": 28,
                          "curve_cpn": "fondeo", "curve_disc": "tiie"},

    # ------------------------- Futuros ------------------------------
    # USD/MXN: incorpora strike y TTM para sensibilidad correcta
    "fut_USDMXN_long_0": {"type": "fut_fx", "under": "MXN=X",
                        "contracts": 100.0, "K": 18.53, "ttm_days": 60,
                        "mult": USDMXN_FUT_MULT, "curve_dom": "dirs", "curve_for": "ones"},
    "fut_USDMXN_long_l": {"type": "fut_fx", "under": "MXN=X",
                        "contracts": 100.0, "K": 18.53, "ttm_days": 60,
                        "mult": USDMXN_FUT_MULT, "curve_dom": "dirs", "curve_for": "libor"},

    "fut_USDMXN_long_f": {"type": "fut_fx", "under": "MXN=X",
                        "contracts": 100.0, "K": 18.53, "ttm_days": 60,
                        "mult": USDMXN_FUT_MULT, "curve_dom": "dirs", "curve_for": "fwd"},


    # WALMEX short:

    "fut_WALMEX_short_0": {"type": "fut_eq", "under": "WALMEX.MX",
                         "contracts": -50.0, "K": 105.0, "ttm_days": 53,
                         "mult": WALMEX_FUT_MULT, "curve_dom": "ones"},

    "fut_WALMEX_short_t": {"type": "fut_eq", "under": "WALMEX.MX",
                         "contracts": -50.0, "K": 105.0, "ttm_days": 53,
                         "mult": WALMEX_FUT_MULT, "curve_dom": "tiie"},

    # WALMEX short:
    "fut_WALMEX_short_y": {"type": "fut_eq", "under": "WALMEX.MX",
                         "contracts": -50.0, "K": 105.0, "ttm_days": 53,
                         "mult": WALMEX_FUT_MULT, "curve_dom": "yield"},

    # ------------------------- Swaps TIIE ---------------------------
    # Paga fijo 11.1% (28d) y recibe TIIE — largo — 588 días
    "swap_pay_fix_long_dsw": {"type": "irs", "notional": 160_000_000.0,
                          "ttm_days": 588, "freq_days": 28, "curve": "dirs_sw",
                          "pay_fixed": True, "K": 0.111},

    "swap_pay_fix_long_tsw": {"type": "irs", "notional": 160_000_000.0,
                          "ttm_days": 588, "freq_days": 28, "curve": "tiie_sw",
                          "pay_fixed": True, "K": 0.111},
    # Recibe fijo 10.9% y paga TIIE — corto — 270 días
    # (notional negativo para reflejar la posición "short")
    "swap_rec_fix_short_dsw": {"type": "irs", "notional": -120_000_000.0,
                           "ttm_days": 270, "freq_days": 28, "curve": "dirs_sw",
                           "pay_fixed": False, "K": 0.109},
    "swap_rec_fix_short_tsw": {"type": "irs", "notional": -120_000_000.0,
                           "ttm_days": 270, "freq_days": 28, "curve": "tiie_sw",
                           "pay_fixed": False, "K": 0.109},

    # ------------------------- Opciones de tasa ---------------------
    # Caplet (CALL) largo — lo reportaremos como "Opción Larga" en la tabla
    "caplet_call_170d_t": {"type": "capfloor_1f", "style": "call",
                         "K": 0.109, "notional": 200_000_000.0,
                         "start_days": 170, "tau_days": 28, "curve": "tiie",
                         "report_bucket": "Opción Larga"},

    "caplet_call_170d_f": {"type": "capfloor_1f", "style": "call",
                         "K": 0.109, "notional": 200_000_000.0,
                         "start_days": 170, "tau_days": 28, "curve": "fondeo",
                         "report_bucket": "Opción Larga"},

    "caplet_call_170d_v": {"type": "capfloor_1f", "style": "call",
                         "K": 0.109, "notional": 200_000_000.0,
                         "start_days": 170, "tau_days": 28, "curve": "tvoltiie_opc",
                         "report_bucket": "Opción Larga"},

    # Floor (PUT) largo — para la plantilla, lo mostraremos en el bloque "Opción Corta"
    # (solo etiqueta de reporte; la valuación sigue siendo posición larga)
    "floor_put_184d_t"  : {"type": "capfloor_1f", "style": "put",
                         "K": 0.110, "notional": 500_000_000.0,
                         "start_days": 184, "tau_days": 28, "curve": "tiie",
                         "report_bucket": "Opción Corta"},
    "floor_put_184d_f"  : {"type": "capfloor_1f", "style": "put",
                         "K": 0.110, "notional": 500_000_000.0,
                         "start_days": 184, "tau_days": 28, "curve": "fondeo",
                         "report_bucket": "Opción Corta"},
    "floor_put_184d_v"  : {"type": "capfloor_1f", "style": "put",
                         "K": 0.110, "notional": 500_000_000.0,
                         "start_days": 184, "tau_days": 28, "curve": "tvoltiie_opc",
                         "report_bucket": "Opción Corta"},
}


In [104]:
# ============================================================
# CUADRO 4: PICKERS DE CURVAS Y VALUACIÓN DE INSTRUMENTOS
# ============================================================

def pick_curve(curve_name: str):
    if curve_name == "guber":
        return curve_guber
    if curve_name == "dirs":
        return curve_dirs
    if curve_name == "libor":
        return curve_libor
    if curve_name == "fondeo":
        return curve_fondeo
    if curve_name == "tiie":
        return curve_tiie
    if curve_name == "fwd":
        return curve_fwd
    if curve_name == "tiie_sw":
        return curve_tiie_sw
    if curve_name == "dirs_sw":
        return curve_dirs_sw
    if curve_name == "yield":
        return curve_yield
    if curve_name == "guber_st":
        return curve_guber_st
    if curve_name == "tvoltiie_opc":
        return curve_tvoltiie_opc
    if curve_name == "ones":
        return curve_ones
    raise ValueError(f"Curva desconocida: {curve_name}")

# -----------------------------
# 4.1 Acciones / FX / BTC (nivel)
# -----------------------------
def pv_equity(qty: float, spot: float) -> float:
    return float(qty * spot)

def pv_fx_units(qty: float, spot_mxn_per_ccy: float) -> float:
    # qty en divisa (USD o EUR), spot es MXN por divisa
    return float(qty * spot_mxn_per_ccy)

def pv_crypto_units(qty: float, spot_mxn: float) -> float:
    return float(qty * spot_mxn)

# -----------------------------
# 4.2 Bono cupón cero MXN
# -----------------------------
def pv_zero_coupon_mxn(notional: float, ttm_days: int, when: pd.Timestamp) -> float:
    df = alambrada_df_func(curve_guber, when)
    return float(notional * df(ttm_days))

# -----------------------------
# 4.3 Bono fijo (tipo Bono M simple semi 182d)
# -----------------------------
def pv_fixed_coupon_mxn(notional: float, coupon_rate: float, ttm_days: int, freq_days: int, when: pd.Timestamp) -> float:
    df = alambrada_df_func(curve_guber, when)
    # paga cupon = notional * coupon_rate * (freq_days/360)
    tau = freq_days / DAY_COUNT
    N = max(1, int(np.ceil(ttm_days / freq_days)))
    pv = 0.0
    for k in range(1, N+1):
        t_k = min(k*freq_days, ttm_days)
        c_k = notional * coupon_rate * tau
        df_k = df(t_k)
        pv += c_k * df_k
    # principal al vencimiento
    pv += notional * df(ttm_days)
    return float(pv)

# -----------------------------
# 4.4 BONDES (flotante 28d): cupón por fondeo y descuesto por curva_disc
# -----------------------------
def pv_bonde(notional: float, ttm_days: int, cpn_freq_days: int, curve_disc_nm: str, curve_cpn_nm: str, when: pd.Timestamp) -> float:
    df_disc = alambrada_df_func(pick_curve(curve_disc_nm), when)
    # cupón forward simple por tramo de 28d de curva_cpn:
    curve_cpn = pick_curve(curve_cpn_nm)
    tau = cpn_freq_days / DAY_COUNT
    N = max(1, int(np.ceil(ttm_days / cpn_freq_days)))
    pv = 0.0
    for k in range(1, N+1):
        t0 = (k-1)*cpn_freq_days
        tk = min(k*cpn_freq_days, ttm_days)
        fwd = forward_rate_from_curve(curve_cpn, when, t0, tk - t0)  # simple
        c_k = notional * fwd * tau
        pv += c_k * df_disc(tk)
    pv += notional * df_disc(ttm_days)
    return float(pv)

# -----------------------------
# 4.5 Futuros: FX y Equity (usar forward teórico y strike)
# -----------------------------
def forward_fx_usdmxn(spot_usdmxn: float, r_dom_curve: pd.DataFrame, r_for_curve: pd.DataFrame,
                      when: pd.Timestamp, T_days: int) -> float:
    # F = S * (D_for / D_dom)^-1  con descuento simple (equivalente: S * (1 + (r_dom - r_for)*T))
    D_dom = alambrada_df_func(r_dom_curve, when)(T_days)
    D_for = alambrada_df_func(r_for_curve, when)(T_days)
    return float(spot_usdmxn * (D_dom / D_for))

def forward_eq(spot: float, r_dom_curve: pd.DataFrame, when: pd.Timestamp, T_days: int, q_div=0.0) -> float:
    # Sin dividendos: F = S / D_dom(T) ~ S*(1 + r*T)
    D_dom = alambrada_df_func(r_dom_curve, when)(T_days)
    return float(spot / D_dom)

def pv_future_fx(contracts: float, mult: float, K: float, T_days: int, spot_usdmxn: float, when: pd.Timestamp) -> float:
    F = forward_fx_usdmxn(spot_usdmxn, curve_dirs, curve_libor, when, T_days)  # dom=MXN (DIRS), for=USD (Libor)
    # MTM aproximado: valor ~ (F - K) * mult * contracts  (sin descuento, por ser mark-to-market diario)
    return float(contracts * mult * (F - K))

def pv_future_eq(contracts: float, mult: float, K: float, T_days: int, spot_eq: float, when: pd.Timestamp) -> float:
    F = forward_eq(spot_eq, curve_guber, when, T_days)
    return float(contracts * mult * (F - K))

# -----------------------------
# 4.6 IRS (Swap) usando fórmula PV ~ (S - K) * Anualidad * Notional  (signo por posición)
# -----------------------------
def par_swap_rate(curve: pd.DataFrame, when: pd.Timestamp, T_days: int, freq_days: int) -> float:
    df = alambrada_df_func(curve, when)
    N = max(1, int(np.ceil(T_days / freq_days)))
    ann = 0.0
    for k in range(1, N+1):
        t_k = min(k*freq_days, T_days)
        ann += df(t_k) * (freq_days / DAY_COUNT)
    D_T = df(T_days)
    S = (1.0 - D_T) / max(ann, 1e-12)
    return float(S)

def pv_swap_tiie(notional: float, K: float, T_days: int, freq_days: int, curve_nm: str, pay_fixed: bool, when: pd.Timestamp) -> float:
    curve = pick_curve(curve_nm)
    S = par_swap_rate(curve, when, T_days, freq_days)
    df = alambrada_df_func(curve, when)
    # Anualidad
    N = max(1, int(np.ceil(T_days / freq_days)))
    ann = 0.0
    for k in range(1, N+1):
        t_k = min(k*freq_days, T_days)
        ann += df(t_k) * (freq_days / DAY_COUNT)
    pv = notional * (S - K) * ann
    # Si somos payer fijo (pagamos K y recibimos float), nuestro PV = (S - K)*A*N
    # Si recibimos fijo, PV = (K - S)*A*N
    if not pay_fixed:
        pv = -pv
    return float(pv)

# -----------------------------
# 4.7 Caplet/Floorlet 1-flujo (Black-76), vol estimada de histórico de la curva
# -----------------------------
def estimate_rate_vol_annual(curve: pd.DataFrame, tenor_days: int, end_date: pd.Timestamp, lookback_days=252) -> float:
    """
    Vol anual a partir de cambios diarios de la tasa a tenor cercano (por alambrada en cada fecha).
    Aproximamos buscando la columna más cercana al tenor si existe; si no, usamos alambrada contra columnas disponibles.
    """
    # series por fecha de la tasa alambrada al tenor solicitado
    tenors = np.array(curve.columns, dtype=int)
    # si hay tenor exacto
    if tenor_days in curve.columns:
        s = curve[tenor_days].copy()
    else:
        # construir a mano por fecha usando alambrada sobre columnas existentes
        def interp_row(row):
            row = row.dropna()
            return alambrada_rate_at(row, tenor_days)
        s = curve.apply(interp_row, axis=1)
    s = s.dropna()
    s = s.loc[:end_date].tail(lookback_days+2).pct_change().dropna()  # cambios relativos (proxy)
    if len(s) < 10:
        return 0.01
    sigma_daily = float(np.std(s, ddof=1))
    return float(sigma_daily * sqrt(252.0))

def black76_capfloor_1f(style: str, K: float, notional: float, start_days: int, tau_days: int,
                        curve_nm: str, when: pd.Timestamp) -> float:
    """
    Precio de caplet/floorlet simple: DF * N * tau * Black76(F, K, sigma, T)
    F forward simple de [start, start+tau] usando curva_nm (DIRS).
    sigma anual estimada de histórico de la misma curva al tenor ~ start_days.
    """
    curve = pick_curve(curve_nm)
    F = forward_rate_from_curve(curve, when, start_days, tau_days)
    DF = alambrada_df_func(curve, when)(start_days + tau_days)
    tau = tau_days / DAY_COUNT

    sigma = max(1e-4, estimate_rate_vol_annual(curve, max(start_days, 28), when))
    T = max(start_days / DAY_COUNT, 1e-6)

    if F <= 0 or not isfinite(F):
        return 0.0

    vol = sigma * sqrt(T)
    if vol < 1e-8:
        # sin volatilidad => payoff determinista
        payoff = max(F - K, 0.0) if style.lower() == "call" else max(K - F, 0.0)
        return float(DF * notional * tau * payoff)

    d1 = (log(F / K) + 0.5 * vol**2) / max(vol, 1e-12)
    d2 = d1 - vol
    if style.lower() == "call":
        price = DF * notional * tau * (F * norm.cdf(d1) - K * norm.cdf(d2))
    else:
        price = DF * notional * tau * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    return float(price)



In [105]:
# ============================================================
# CUADRO 5: VALORACIÓN "HOY" (VAL_DATE) DE CADA INSTRUMENTO
# ============================================================

def get_spot_today(px_panel: pd.DataFrame, ticker: str) -> float:
    s = px_panel[ticker].dropna()
    last = _last_business_before(VAL_DATE, s)
    return float(s.loc[last])

def value_today(name: str, inst: dict, px_panel: pd.DataFrame) -> float:
    t = inst["type"]

    if t == "equity":
        S0 = get_spot_today(px_panel, inst["ticker"])
        return pv_equity(inst["qty"], S0)

    if t == "fx":
        S0 = get_spot_today(px_panel, inst["ticker"])
        return pv_fx_units(inst["qty"], S0)

    if t == "crypto":
        S0 = get_spot_today(px_panel, inst["ticker"])
        return pv_crypto_units(inst["qty"], S0)

    if t == "bond_zc":
        return pv_zero_coupon_mxn(inst["notional"], inst["ttm_days"], VAL_DATE)

    if t == "bond_fix":
        return pv_fixed_coupon_mxn(inst["notional"], inst["coupon"], inst["ttm_days"], inst["freq_days"], VAL_DATE)

    if t == "bonde":
        return pv_bonde(inst["notional"], inst["ttm_days"], inst["coupon_freq_days"], inst["curve_disc"], inst["curve_cpn"], VAL_DATE)

    if t == "fut_fx":
        S0 = get_spot_today(px_panel, inst["under"])
        return pv_future_fx(inst["contracts"], inst["mult"], inst["K"], inst["ttm_days"], S0, VAL_DATE)

    if t == "fut_eq":
        S0 = get_spot_today(px_panel, inst["under"])
        return pv_future_eq(inst["contracts"], inst["mult"], inst["K"], inst["ttm_days"], S0, VAL_DATE)

    if t == "irs":
        return pv_swap_tiie(inst["notional"], inst["K"], inst["ttm_days"], inst["freq_days"], inst["curve"], inst["pay_fixed"], VAL_DATE)

    if t == "capfloor_1f":
        return black76_capfloor_1f(inst["style"], inst["K"], inst["notional"], inst["start_days"], inst["tau_days"], inst["curve"], VAL_DATE)

    raise ValueError(f"Tipo no soportado: {t}")




# Construir precios históricos y valorar hoy
PX = build_spot_panel()
VALS_TODAY = pd.Series({nm: value_today(nm, it, PX) for nm, it in PORT.items()})
VAL_PTF_TODAY = float(VALS_TODAY.sum())

print("Valor hoy por instrumento (MXN):")
display(VALS_TODAY.round(2))
print(f"\nValor total portafolio (MXN) al {VAL_DATE_STR}: {VAL_PTF_TODAY:,.2f}")






Valor hoy por instrumento (MXN):


,0
eq_GCARSO,128149.99
eq_AMXB,-183899.99
eq_WALMEX,170400.00
fx_USD,28089.54
fx_EUR,26318.64
cr_BTC,2801.79
bono_CETES_ZC_180,14790.67
bonoM_11p5_3600,2330.93
bondes_1200_707d_g,-1429.50
bondes_1200_707d_f,-1206.05



Valor total portafolio (MXN) al 2025-09-08: 2,985,770.83


In [106]:
# ============================================================
# CUADRO 6: ESCENARIOS HISTÓRICOS (RETORNOS n-DÍAS) Y ALINEACIÓN
# ============================================================

def hist_returns_panel(px_panel: pd.DataFrame, n_days: int) -> pd.DataFrame:
    rets = px_panel.apply(lambda s: make_returns_for_horizon(s, n_days))
    return rets.dropna()

def curve_shift(curve: pd.DataFrame, n_days: int) -> pd.DataFrame:
    """
    Cambios de tasas (Δ y) n-días: y_t - y_{t-n}. Devuelve Δy por tenor y fecha t.
    Estos Δ se aplican aditivamente a la curva "hoy" para generar curva de escenario.
    """
    return (curve - curve.shift(n_days)).dropna()

def intersect_dates(*dfs):
    idx = None
    for d in dfs:
        idx = d.index if idx is None else idx.intersection(d.index)
    return idx

# Construir shocks históricos
rets = hist_returns_panel(PX, HORIZON_N_DAYS)
dG = curve_shift(curve_guber, HORIZON_N_DAYS)
dD = curve_shift(curve_dirs,  HORIZON_N_DAYS)
dL = curve_shift(curve_libor, HORIZON_N_DAYS)
dF = curve_shift(curve_fondeo, HORIZON_N_DAYS)

common_idx = intersect_dates(rets, dG, dD, dL, dF)
rets = rets.loc[common_idx]
dG, dD, dL, dF = dG.loc[common_idx], dD.loc[common_idx], dL.loc[common_idx], dF.loc[common_idx]

print(f"Escenarios listos (n={HORIZON_N_DAYS} días): {len(common_idx)} fechas.")


Escenarios listos (n=1 días): 254 fechas.


In [107]:
# @title
# ============================================================
# CUADRO 7: VALUACIÓN POR ESCENARIO Y MATRIZ DE P&L
# ============================================================

def curve_today_row(curve: pd.DataFrame, when: pd.Timestamp) -> pd.Series:
    # Selecciona fila de curva más reciente <= when
    if when in curve.index:
        return curve.loc[when].dropna()
    pos = curve.index.searchsorted(when)
    pos = max(0, pos - 1)
    return curve.iloc[pos].dropna()

def _apply_curve_shock_today(curve: pd.DataFrame, dY_row: pd.Series) -> pd.Series:
    """
    Suma aditivamente Δy a la curva 'hoy' por tenor. Si falta algún tenor en Δy se ignora (deja 'hoy').
    """
    base = curve_today_row(curve, VAL_DATE).copy()
    for ten in dY_row.index.intersection(base.index):
        base.loc[ten] = base.loc[ten] + dY_row.loc[ten]
    return base

def df_from_row(row: pd.Series):
    def df(t_days: int) -> float:
        r_pct = alambrada_rate_at(row, max(int(t_days), 1))
        r = r_pct / 100.0
        t = max(t_days, 0) / DAY_COUNT
        return 1.0 / (1.0 + r * t)
    return df

def value_in_scenario(name: str, inst: dict, px0: pd.DataFrame, rets_row: pd.Series,
                      dG_row: pd.Series, dD_row: pd.Series, dL_row: pd.Series, dF_row: pd.Series) -> float:
    t = inst["type"]

    # Spots hoy
    def S_today(ticker): return get_spot_today(px0, ticker)

    # Spots shockeados
    def S_scn(ticker):
        r = rets_row.get(ticker, np.nan)
        if np.isnan(r):
            # BTCMXN no tiene retorno directo si no está en columnas; construimos si hace falta
            if ticker == "BTCMXN":
                r_btc = rets_row.get(TICK_BTCUSD, np.nan)
                r_usd = rets_row.get(TICK_USDMXN, np.nan)
                r = (1.0 + (0 if np.isnan(r_btc) else r_btc)) * (1.0 + (0 if np.isnan(r_usd) else r_usd)) - 1.0
            else:
                r = 0.0
        return apply_price_shock(S_today(ticker), r)

    # Curvas shockeadas (sumas aditivas a la curva de hoy)
    rowG = _apply_curve_shock_today(curve_guber, dG_row)
    rowD = _apply_curve_shock_today(curve_dirs,  dD_row)
    rowL = _apply_curve_shock_today(curve_libor, dL_row)
    rowF = _apply_curve_shock_today(curve_fondeo, dF_row)
    rowT = _apply_curve_shock_today(curve_tiie, dF_row)
    rowY = _apply_curve_shock_today(curve_yield, dF_row)
    rowS = _apply_curve_shock_today(curve_fwd, dF_row)
    rowV = _apply_curve_shock_today(curve_tvoltiie_opc, dF_row)
    rowI = _apply_curve_shock_today(curve_dirs_sw, dF_row)
    rowJ = _apply_curve_shock_today(curve_tiie_sw, dF_row)
    rowK = _apply_curve_shock_today(curve_guber_st, dF_row)


    # Helpers DF y forward con curvas de escenario
    DF_G = df_from_row(rowG)
    DF_D = df_from_row(rowD)
    DF_L = df_from_row(rowL)
    DF_F = df_from_row(rowF)
    DF_T = df_from_row(rowT)
    DF_Y = df_from_row(rowY)
    DF_S = df_from_row(rowS)
    DF_V = df_from_row(rowV)
    DF_I = df_from_row(rowI)
    DF_J = df_from_row(rowJ)
    DF_K = df_from_row(rowK)

    def fwd_rate(row, start_d, tau_d):
        D = df_from_row(row)
        t0, t1 = start_d, start_d + tau_d
        return (D(t0)/D(t1) - 1.0)/max((t1 - t0)/DAY_COUNT, 1e-9)

    # ===== Instrumentos =====
    if t == "equity":
        return pv_equity(inst["qty"], S_scn(inst["ticker"]))

    if t == "fx":
        return pv_fx_units(inst["qty"], S_scn(inst["ticker"]))

    if t == "crypto":
        return pv_crypto_units(inst["qty"], S_scn(inst["ticker"]))

    if t == "bond_zc":
        return float(inst["notional"] * DF_G(inst["ttm_days"]))

    if t == "bond_fix":
        tau = inst["freq_days"]/DAY_COUNT
        N = max(1, int(np.ceil(inst["ttm_days"]/inst["freq_days"])))
        pv = 0.0
        for k in range(1, N+1):
            tk = min(k*inst["freq_days"], inst["ttm_days"])
            ck = inst["notional"] * inst["coupon"] * tau
            pv += ck * DF_G(tk)
        pv += inst["notional"] * DF_G(inst["ttm_days"])
        return float(pv)

    if t == "bonde":
        tau = inst["coupon_freq_days"]/DAY_COUNT
        N = max(1, int(np.ceil(inst["ttm_days"]/inst["coupon_freq_days"])))
        pv = 0.0
        for k in range(1, N+1):
            t0 = (k-1)*inst["coupon_freq_days"]
            tk = min(k*inst["coupon_freq_days"], inst["ttm_days"])
            fwd = fwd_rate(rowF, t0, tk - t0)  # cupón por fondeo
            ck = inst["notional"] * fwd * tau
            pv += ck * DF_G(tk)               # descuento por guber (o cambiar a DF_F si así lo prefieres)
        pv += inst["notional"] * DF_G(inst["ttm_days"])
        return float(pv)

    if t == "fut_fx":
        S = S_scn(inst["under"])
        # F_scn con curvas de escenario:
        F = S * (DF_D(inst["ttm_days"]) / DF_L(inst["ttm_days"]))
        return float(inst["contracts"] * inst["mult"] * (F - inst["K"]))

    if t == "fut_eq":
        S = S_scn(inst["under"])
        F = S / DF_G(inst["ttm_days"])
        return float(inst["contracts"] * inst["mult"] * (F - inst["K"]))

    if t == "irs":
        # Swap con curvas shockeadas (usando par rate S y anualidad sobre DF_D)
        # Par rate
        Np = max(1, int(np.ceil(inst["ttm_days"]/inst["freq_days"])))
        ann = 0.0
        for k in range(1, Np+1):
            tk = min(k*inst["freq_days"], inst["ttm_days"])
            ann += DF_D(tk) * (inst["freq_days"]/DAY_COUNT)
        S_par = (1.0 - DF_D(inst["ttm_days"])) / max(ann, 1e-12)
        pv = inst["notional"] * (S_par - inst["K"]) * ann
        if not inst["pay_fixed"]:
            pv = -pv
        return float(pv)

    if t == "capfloor_1f":
        # Black-76 con curvas shockeadas y sigma reutilizando la estimación histórica "hoy"
        # (para estabilidad, no re-estimamos sigma por escenario)
        # Necesitamos F y DF del escenario:
        t0 = inst["start_days"]
        tau = inst["tau_days"]
        D0, D1 = DF_D(t0), DF_D(t0+tau)
        F = (D0 / D1 - 1.0) / (tau / DAY_COUNT)
        T = max(t0 / DAY_COUNT, 1e-6)
        DF_pay = D1
        sigma = max(1e-4, estimate_rate_vol_annual(curve_dirs, max(t0,28), VAL_DATE))

        vol = sigma * sqrt(T)
        if vol < 1e-8 or F <= 0:
            payoff = max(F - inst["K"], 0.0) if inst["style"]=="call" else max(inst["K"] - F, 0.0)
            return float(DF_pay * inst["notional"] * (tau/DAY_COUNT) * payoff)
        d1 = (log(F / inst["K"]) + 0.5 * vol**2) / max(vol, 1e-12)
        d2 = d1 - vol
        if inst["style"] == "call":
            price = DF_pay * inst["notional"] * (tau/DAY_COUNT) * (F*norm.cdf(d1) - inst["K"]*norm.cdf(d2))
        else:
            price = DF_pay * inst["notional"] * (tau/DAY_COUNT) * (inst["K"]*norm.cdf(-d2) - F*norm.cdf(-d1))
        return float(price)

    raise ValueError(f"Tipo no soportado: {t}")

# ---------- Construcción de matrices ----------
names = list(PORT.keys())

# Valores por escenario
vals_scen = []
for dt, row in rets.iterrows():
    v = {}
    dG_row = dG.loc[dt]
    dD_row = dD.loc[dt]
    dL_row = dL.loc[dt]
    dF_row = dF.loc[dt]
    for nm in names:
        v[nm] = value_in_scenario(nm, PORT[nm], PX, row, dG_row, dD_row, dL_row, dF_row)
    vals_scen.append(pd.Series(v, name=dt))

VALS_SCEN = pd.DataFrame(vals_scen).sort_index()
PVAL_SCEN = VALS_SCEN.sum(axis=1)

# P&L por escenario
P0 = VALS_TODAY
PNL_SCEN = VALS_SCEN.sub(P0, axis=1)
PNL_PTF = PVAL_SCEN - VAL_PTF_TODAY

print("Listo: matrices de valoración y P&L por escenario construidas.")



Listo: matrices de valoración y P&L por escenario construidas.


In [108]:
# ============================================================
# CUADRO 8: VaR / CVaR (HISTÓRICO PURO) Y EWMA + MARGINALES
# ============================================================

def var_cvar_series(losses: pd.Series, alpha=ALPHA, weights=None):
    """
    Calcula VaR y CVaR (Expected Shortfall) de una serie de pérdidas (+ = pérdida).
    - losses: pd.Series indexado por fecha
    - weights: np.array de igual longitud (si None, no ponderado)
    """
    x = losses.values.copy()
    if weights is None:
        # no ponderado
        q = np.quantile(x, alpha, method="linear")
        tail = x[x >= q]
        cvar = float(tail.mean())
        return float(q), float(cvar)
    else:
        w = np.asarray(weights, dtype=float)
        w = w / w.sum()
        # cuantíl ponderado
        order = np.argsort(x)
        xs, ws = x[order], w[order]
        cws = np.cumsum(ws)
        # primer índice con cws >= alpha
        idx = np.searchsorted(cws, alpha)
        q = float(xs[min(idx, len(xs)-1)])
        # CVaR ponderado = media de cola por pesos renormalizados en cola
        tail_mask = xs >= q
        wt = ws[tail_mask]
        xt = xs[tail_mask]
        if wt.sum() > 0:
            cvar = float(np.sum(wt * xt) / wt.sum())
        else:
            cvar = q
        return float(q), float(cvar)

def ewma_weights(n: int, lam=EWMA_LAMBDA):
    w = np.array([lam**(n-1-i) for i in range(n)], dtype=float)
    return w / w.sum()

# PÉRDIDAS (convención: pérdida = -PNL si PNL<0 => pérdida positiva)
LOSS_PTF = -PNL_PTF
LOSS_BY_ASSET = -PNL_SCEN  # columna por instrumento

# No alisado
VaR_ptf, CVaR_ptf = var_cvar_series(LOSS_PTF, ALPHA, weights=None)

# EWMA (más peso a recientes)
w_ewma = ewma_weights(len(LOSS_PTF), EWMA_LAMBDA)
VaR_ptf_E, CVaR_ptf_E = var_cvar_series(LOSS_PTF, ALPHA, weights=w_ewma)

# Por instrumento (standalone)
VaR_by = {}
CVaR_by = {}
VaR_by_E = {}
CVaR_by_E = {}
for nm in names:
    l = LOSS_BY_ASSET[nm]
    VaR_by[nm], CVaR_by[nm] = var_cvar_series(l, ALPHA, None)
    VaR_by_E[nm], CVaR_by_E[nm] = var_cvar_series(l, ALPHA, w_ewma)

# Contribuciones marginales (ES coherente, estilo Euler): E[ loss_i | loss_ptf >= VaR ]
q_port, _ = var_cvar_series(LOSS_PTF, ALPHA, None)
tail_mask = LOSS_PTF >= q_port
ES_contrib = LOSS_BY_ASSET[tail_mask].mean(axis=0)  # standalone contrib condicional
# Normalizar a que sumen ES del portafolio
scale = CVaR_ptf / max(ES_contrib.sum(), 1e-12)
ES_contrib = ES_contrib * scale

# Lo mismo con EWMA: usar promedio ponderado en cola
q_port_E, _ = var_cvar_series(LOSS_PTF, ALPHA, w_ewma)
tail_mask_E = LOSS_PTF >= q_port_E
w_tail = w_ewma[tail_mask_E]
if w_tail.sum() > 0:
    ES_contrib_E = (LOSS_BY_ASSET[tail_mask_E].T.dot(w_tail) / w_tail.sum())
    ES_contrib_E = pd.Series(ES_contrib_E, index=names)
    scaleE = CVaR_ptf_E / max(ES_contrib_E.sum(), 1e-12)
    ES_contrib_E = ES_contrib_E * scaleE
else:
    ES_contrib_E = pd.Series(0.0, index=names)

# OUTPUTS
print(f"== VaR/CVaR Portafolio (α={ALPHA:.2%}, horizonte={HORIZON_N_DAYS} día(s)) ==")
print(f"  Sin alisado:  VaR = {VaR_ptf:,.2f}  |  CVaR = {CVaR_ptf:,.2f}")
print(f"  Con EWMA:     VaR = {VaR_ptf_E:,.2f} |  CVaR = {CVaR_ptf_E:,.2f}")

print("\n== Standalone (por instrumento) — Sin alisado ==")
display(pd.DataFrame({"VaR": VaR_by, "CVaR": CVaR_by}).round(2))

print("== Standalone (por instrumento) — Con EWMA ==")
display(pd.DataFrame({"VaR_EWMA": VaR_by_E, "CVaR_EWMA": CVaR_by_E}).round(2))

print("== Contribución marginal a CVaR (Euler) — Sin alisado ==")
display(ES_contrib.round(2))

print("== Contribución marginal a CVaR (Euler) — Con EWMA ==")
display(ES_contrib_E.round(2))


== VaR/CVaR Portafolio (α=99.00%, horizonte=1 día(s)) ==
  Sin alisado:  VaR = 7,979,793.31  |  CVaR = 8,232,181.57
  Con EWMA:     VaR = 8,061,758.67 |  CVaR = 8,061,760.77

== Standalone (por instrumento) — Sin alisado ==


,VaR,CVaR
eq_GCARSO,6003.99,6232.52
eq_AMXB,5172.30,6749.81
eq_WALMEX,6992.51,8764.19
fx_USD,424.48,479.02
fx_EUR,341.63,395.47
cr_BTC,173.30,206.12
bono_CETES_ZC_180,0.11,0.17
bonoM_11p5_3600,0.59,0.72
bondes_1200_707d_g,9.24,12.88
bondes_1200_707d_f,232.69,236.33


== Standalone (por instrumento) — Con EWMA ==


,VaR_EWMA,CVaR_EWMA
eq_GCARSO,3969.91,4328.78
eq_AMXB,3448.11,3511.19
eq_WALMEX,3663.94,3788.41
fx_USD,530.40,530.40
fx_EUR,472.72,472.72
cr_BTC,150.45,163.63
bono_CETES_ZC_180,0.11,0.11
bonoM_11p5_3600,0.67,0.72
bondes_1200_707d_g,1.30,1.35
bondes_1200_707d_f,224.75,224.80


== Contribución marginal a CVaR (Euler) — Sin alisado ==


,0
eq_GCARSO,-1835.78
eq_AMXB,459.78
eq_WALMEX,963.75
fx_USD,305.20
fx_EUR,3.22
cr_BTC,-40.16
bono_CETES_ZC_180,-0.00
bonoM_11p5_3600,0.28
bondes_1200_707d_g,0.34
bondes_1200_707d_f,223.79


== Contribución marginal a CVaR (Euler) — Con EWMA ==


,0
eq_GCARSO,1558.78
eq_AMXB,1114.54
eq_WALMEX,1239.08
fx_USD,400.58
fx_EUR,222.79
cr_BTC,63.27
bono_CETES_ZC_180,-0.00
bonoM_11p5_3600,0.53
bondes_1200_707d_g,0.14
bondes_1200_707d_f,223.59
